In [1]:
!python ../setup.py build 
!python ../setup.py build_ext --inplace

First: /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot
/Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot
running build
running build_py
file /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/opttrot.py (for module opttrot) not found
file /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/opttrot.py (for module opttrot) not found
running build_ext
building 'pauli_c' extension
clang -fno-strict-overflow -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /Users/hyunseongkim/miniconda3/envs/opttrot/include -arch arm64 -fPIC -O2 -isystem /Users/hyunseongkim/miniconda3/envs/opttrot/include -arch arm64 -DBIG_NUM_BYTES=512 -I/Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot -I/Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/c_src -I/Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptTrot/opttrot/c_src/bn -I/Users/hyunseongkim/miniconda3/envs/opttrot/include/python3.12 -c /Users/hyunseongkim/Documents/GitHub/Org_OptTrot/OptT

In [2]:
import os, sys
sys.path.append("..")

# Qiskit 1.1.1


In [1]:
import os, sys
sys.path.append("..")

In [2]:
from opttrot.pauli import PauliElement
from opttrot.pauli import PauliPoly
from opttrot.pauli_utils import get_pstrs, pstr2sym_code

from importlib import reload
import ten_con
reload(ten_con)
from ten_con import _mat_to_coef_mat, _coef_to_mat

import numpy as np
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Pauli

from copy import deepcopy

In [3]:
pl = []
pl_qiskit = []
n = 10
pstrs = get_pstrs(n)
l = len(pstrs)
reps = 1
pstrs = pstrs[:int(l/reps)]
for p in pstrs:
    nx, nz = pstr2sym_code(p)
    pl.append(PauliElement(nx=nx, nz=nz, n =n, weight=1))
    pl_qiskit.append(Pauli(p))

In [4]:
sample_ppoly = PauliPoly(pl)
pmat = sample_ppoly.matrix
cmat = sample_ppoly.coef_matrix

In [5]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [6]:
pmat2 = deepcopy(pmat)

### Decomposition

Qiskit method is a tensorized decomposition of hermit matrix into pauli terms.
See Hantzko et al, 2023.

|          |Qiksit|OptTrot|
|:--------:|:----:|:-----:|
|Language| Rust| Python+Numba(JIT)|
| Method| TPD| TPD|

- Qiskit: Decompose and element construct in Rust -> return Python.
- OptTrot: Decompose in JIT and construct element in Python(-> Structure is not optimal).

In [9]:
%%timeit
sp1 = SparsePauliOp.from_operator(pmat2) # Using a rust module including parallelism.

288 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
mat = _mat_to_coef_mat(pmat) # Core

252 ms ± 5.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
mat = _mat_to_coef_mat(pmat)

In [8]:
sp = SparsePauliOp.from_operator(pmat2)

In [12]:
%%timeit
omat = sp.to_matrix()

885 ms ± 65.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
cmat2 = deepcopy(cmat)
omat = _coef_to_mat(cmat2)

In [14]:
%%timeit
omat = _coef_to_mat(cmat)

275 ms ± 7.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
cmat = sample_ppoly.coef_matrix # 이 부분의 속도를 개선해보자.
omat = _coef_to_mat(cmat)

1min 3s ± 199 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
zero = np.zeros(cmat.shape, dtype = complex)
for p in pl_qiskit:
    zero += p.to_matrix()

KeyboardInterrupt: 

In [38]:
arr = np.array([(1, 2), (2, 3), (4, 5)], dtype=object)
arr

array([[1, 2],
       [2, 3],
       [4, 5]], dtype=object)

In [40]:
arr = np.array([(1, 2), (2, 3), (4, 5)], dtype=object)
arr.shape

(3, 2)

In [42]:
arr_1d = np.array([tuple(row) for row in arr], dtype=object)
arr_1d

array([[1, 2],
       [2, 3],
       [4, 5]], dtype=object)

In [43]:
arr = np.empty(3, dtype=object)
arr[0] = (1, 2)
arr[1] = (2, 3)
arr[2] = (4, 5)

In [44]:
arr

array([(1, 2), (2, 3), (4, 5)], dtype=object)

In [45]:
result = np.vstack(arr)

In [46]:
result

array([[1, 2],
       [2, 3],
       [4, 5]])